In [114]:
import requests
import pandas as pd
from time import sleep

In [115]:
NFT_CONTRACT = "terra103z9cnqm8psy0nyxqtugg6m7xnwvlkqdzm4s4k"
OFFSET = 158292848

In [69]:
def get_message_from_tx(tx):
    m_list = []
    messages = tx['tx']['value']['msg']
    for m in messages:
        if 'execute_msg' in m['value'].keys():
            msg_obj = {
                'type':m['type'],
                'execute_msg':m['value']['execute_msg']
            }
            m_list.append(msg_obj)
    return m_list

In [98]:
def get_all_messages(nft_contract, offset = 0 ):
    loop = True
    m_list = []
    offset = offset
    while loop:
        request_success = False
        while not request_success:
            data = requests.get(f"https://fcd.terra.dev/v1/txs?offset={offset}&limit=100&account={nft_contract}")
            request_success = True if data.status_code == 200 else False
            if request_success == False:
                sleep(20)
                print(data.text)
                print("retrying request in 20 secs")
        x = data.json()
        for tx in x['txs']:
            m_list = m_list + get_message_from_tx(tx)
        if 'next' in x.keys():
            print(offset)
            offset = x['next']
        else:
            break
        sleep(1)
    m_list = [i for i in m_list if 'mint_nft' in i['execute_msg']]
    return m_list

In [99]:
def generate_dataframe(message_list):
    for i in message_list:
        i['token_id'] = i['execute_msg']['mint_nft']['token_id']
        i['name'] = i['execute_msg']['mint_nft']['extension']['name']
        i['image'] = i['execute_msg']['mint_nft']['extension']['image']
        i['attributes'] = i['execute_msg']['mint_nft']['extension']['attributes']
        i['description'] = i['execute_msg']['mint_nft']['extension']['description']
        i['token_uri'] = i['execute_msg']['mint_nft']['token_uri']
        
    for i in message_list:
        for a in i['attributes']:
            i[a['trait_type']] = a['value']
            
    df = pd.DataFrame(message_list)
    df['number_attributes'] = df.notnull()[
        ['backgrounds','suits','species','face','hair','glasses','headware','jewelry']
    ].sum(axis=1)
    df_lean = df[
        ['description','name','token_id','image','token_uri',
         'backgrounds','suits','species','face','hair','glasses',
         'headware','jewelry','number_attributes']
    ]
    return df_lean

In [100]:
def calculate_rarity(row, attribute_types):
    score = 0
    for attribute in attribute_types:
        score = score + 1 / row[f"{attribute}_probability"]
    return score

In [109]:
def generate_dataframe(messages):
    extensions = [x["execute_msg"]["mint_nft"]["extension"] for x in messages]
    for j in extensions:
        ipfs_id = j["image"].replace("ipfs://","")
        j['ipfs_url'] = f"https://cf-ipfs.com/ipfs/{ipfs_id}"
        for a in j["attributes"]:
            j[a["trait_type"]] = a["value"]
    df = pd.DataFrame(extensions)
    attribute_types = [x["trait_type"] for x in df["attributes"][0]]
    attribute_prob_df_dict = {}
    for attribute in attribute_types:
        attribute_df = df[attribute].value_counts(dropna=False).rename_axis(attribute).reset_index(name=f'{attribute}_count')
        attribute_df[f'{attribute}_probability'] = attribute_df[f'{attribute}_count'] / attribute_df[f'{attribute}_count'].sum()
        attribute_prob_df_dict[attribute] = attribute_df
    for attribute in attribute_types:
        df = df.merge(attribute_prob_df_dict[attribute],how="left",on=attribute)
    df["rarity_score"] = df.apply(lambda x : calculate_rarity(x,attribute_types),axis=1)
    df = df.sort_values("rarity_score",ascending=False)
    df["rarity_rankings"] = range(1,len(df)+1)
    return df

In [111]:
messages = get_all_messages(NFT_CONTRACT,offset=OFFSET)

158292848
158145174
157522765
157504392


In [112]:
df = generate_dataframe(messages)

In [113]:
df.to_csv(f"{NFT_CONTRACT}.csv",index=None)